#### The purpose of this notebook is to demonstrate a notional ML application, indended for deployment via and MLOps pipeline

In [51]:
import pandas as pd
import numpy as np
import sklearn
import random as rand
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from pandas.io.json import json_normalize
from IPython.display import clear_output
import ipywidgets as widgets
import pickle

## Vectice Version

#### 1. Load all of the data and model resources from previous steps

In [142]:
#list of features to be included in the inference step
col_imp_vect = list(X_valid)

In [141]:
with open('data/serial/X_valid.pkl', 'rb') as file:
    # Call load method to deserialze
    X_valid = pickle.load(file)

with open('data/serial/y_valid.pkl', 'rb') as file:
    # Call load method to deserialze
    y_valid = pickle.load(file)

In [143]:
#load the pre-trained model
with open('models/serial/rf_regressor_ver_18', 'rb') as file:
    # Call load method to deserialze
    rf_regressor = pickle.load(file)

#### 2. Define the Prediction function to be used in the UI

In [144]:
def predict(dict_values, col_imp=col_imp, clf=clf):
    x = np.array([float(dict_values[col]) for col in col_imp_vect])
    x = x.reshape(1,-1)
    y_pred = rf_regressor.predict(x)[0]
    y_pred_string = "$"+ str(round(y_pred,2))
    return y_pred_string

#### 3. Build the Inference UI

In [147]:
#define UI
score_btn = widgets.Button(description='Predict Price', 
                           button_style='primary',
                           layout = widgets.Layout(width='auto', height='40px'))
btn_select_random = widgets.Button(description = 'Select Random House', 
                                   layout = widgets.Layout(width='auto', height='auto'))
out_select = widgets.Output(layout={'border': '1px solid black'})
out_score = widgets.Output(layout={'border': '1px solid black'})


widgets.Layout(width = 'auto')

def btn_select_random_eventhandler(obj):
    with out_select:
        global x_test_json
        x_test_json = X_valid[col_imp_vect].iloc[rand.randint(1,2100)].T.replace('},{', '} {')
        clear_output()
        print(x_test_json)
    with out_score:
        clear_output()
          
        
        
def btn_score_eventhandler(obj):
    with out_score:
        clear_output()
        print("Predicted Price: $",predict(x_test_json) )

#### 4. Display the UI and Test

In [148]:
display(widgets.VBox([btn_select_random,out_select,score_btn,out_score]))

btn_select_random.on_click(btn_select_random_eventhandler)
score_btn.on_click(btn_score_eventhandler)

## Original Version

In [87]:
data = pd.read_csv("data/kc_house_data.csv")

In [89]:
x_train, x_test, y_train, y_test = train_test_split(train1, labels, test_size=0.10, random_state=2)


In [93]:
labels = data['price']
conv_dates = [1 if values == 2014 else 0 for values in data.date ]
data['date'] = conv_dates
train1 = data.drop(['id', 'price'],axis=1)

In [90]:
col_imp = ["grade", "lat", "long", "sqft_living", "waterfront", "yr_built"]

In [91]:
clf = GradientBoostingRegressor(n_estimators=400, max_depth=5, min_samples_split=2,
          learning_rate=0.1, loss='squared_error')

In [100]:
clf.fit(x_train[col_imp], y_train)

GradientBoostingRegressor(max_depth=5, n_estimators=400)

In [101]:
def predict(dict_values, col_imp=col_imp, clf=clf):
    x = np.array([float(dict_values[col]) for col in col_imp])
    x = x.reshape(1,-1)
    y_pred = clf.predict(x)[0]
    y_pred_string = "$"+ str(round(y_pred,2))
    return y_pred_string

In [102]:
#define UI
score_btn = widgets.Button(description='Predict Price', 
                           button_style='primary',
                           layout = widgets.Layout(width='auto', height='40px'))
btn_select_random = widgets.Button(description = 'Select Random House', 
                                   layout = widgets.Layout(width='auto', height='auto'))
out_select = widgets.Output(layout={'border': '1px solid black'})
out_score = widgets.Output(layout={'border': '1px solid black'})


widgets.Layout(width = 'auto')

def btn_select_random_eventhandler(obj):
    with out_select:
        global x_test_json
        x_test_json = x_test[col_imp].iloc[rand.randint(1,2100)].T.replace('},{', '} {')
        clear_output()
        print(x_test_json)
    with out_score:
        clear_output()
          
        
        
def btn_score_eventhandler(obj):
    with out_score:
        clear_output()
        print("Predicted Price: $",predict(x_test_json) )

### Render notebook as web app using Viola

In [103]:
display(widgets.VBox([btn_select_random,out_select,score_btn,out_score]))

btn_select_random.on_click(btn_select_random_eventhandler)
score_btn.on_click(btn_score_eventhandler)